In [39]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import csv
import chromedriver_binary
import urllib
import os
import requests
import json

# 以下、グーグルスプレッドシート向け
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

jsonf = "google_spreadsheet_key/melodic-splicer-343106-65436442be5f.json"
# spread_sheet_key = "1Fq-6SzLcxhcDNao4MsDv0RZckvy5cRALNg8N7NkkzJQ"
spread_sheet_key = "11AIaBScIfoTbntWrGueuxUaiWPyE7vaqwjvKu2HMl8w"

In [40]:
def connect_gspread(jsonf,key,sheet_name):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    #worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    worksheet = gc.open_by_key(SPREADSHEET_KEY).worksheet(sheet_name)
    return worksheet

In [30]:
#ws_write = connect_gspread(jsonf,spread_sheet_key,'自動更新')
#ws_read = connect_gspread(jsonf,spread_sheet_key,'02_出店リスト')

In [42]:
def write_csv( file_path, data ):
    with open( file_path, 'a' ) as f:
        print( ','.join( data ), file=f )

def get_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument("--no-sandbox")
    return webdriver.Chrome( options = options )

def get_page_bs( driver, url ):
    driver.get( url )
    time.sleep( 5 ) # 5秒間待機
    return BeautifulSoup( driver.page_source, features='lxml' )

In [43]:
# スプレッドシートからASINを取得
def get_asin():
    res = []
    data = ws_read.get_all_values()
    for i in range( 0 ,len(data) ):
        # col = 9
        if data[i][col] != "" and len(data[i][col]) == 10 and data[i][col] not in res:
            res.append( data[i][col] )
    return res

def r( string ):
    if type( string ) is not str:
        return string
    return string.replace("\n","").replace("\u3000"," ").replace( " ", "" )

In [44]:
# Ajaxページから指定されたページ出品者のデータを収集
def get_url( asin, page_num ):
    return "https://www.amazon.co.jp/gp/product/ajax?asin="+asin+"&pc=dp&isonlyrenderofferlist=true&pageno="+str(page_num)+"&experienceId=aodAjaxMain"

def get_sellers( driver, asin, page_num ):
    print( get_url( asin, page_num ) )
    bs = get_page_bs( d, get_url( asin, page_num ) )
    res = []
    for item in bs.find_all( 'div', id = 'aod-offer' ):
        condition = r( item.find( 'div', id = 'aod-offer-heading' ).text ) #新品か中古か
        price = r( item.find( 'div', id = 'aod-offer-price' ).find( 'span', class_ = 'a-price-whole' ).text )
        seller_data = item.find( 'div', id = 'aod-offer-soldBy' )
        #seller_name = r( seller_data.find( 'a' ).text )
        seller_url = "https://www.amazon.co.jp" + r( seller_data.find( 'a' ).get('href') )
        seller_id = seller_url[ seller_url.find('seller=') + 7 : seller_url.find('seller=') + 21 ]
        if seller_url.find('seller=') < 1:
            # Amazon公式販売なので、、、
            seller_url = "https://www.amazon.co.jp/gp/help/customer/display.html?nodeId=202008070"
            seller_id = ""
            #seller_name = "Amazon.co.jp"
        # res.append( ["",asin, condition, price, seller_id, seller_name, seller_url] )
        res.append( [asin,"",price,seller_id] )
    return res

In [45]:
# Ajaxページから全出品者のデータを収集
def get_all_sellers( driver, asin ):
    res = []
    seller_ids = []
    for i in range(1,100):
        flag = True
        new_sellers = get_sellers( driver, asin, i )
        if len( new_sellers ) == 0:
            break
        for new_seller in new_sellers:
            if new_seller[3] in seller_ids:
                flag = False
                continue
            else:
                res.append( new_seller )
                seller_ids.append( new_seller[3] )
        if not flag:
            return res
    return res

In [46]:
# カート販売者の情報を商品ページから収集
def get_cart_seller( driver, asin ):
    url = "https://www.amazon.co.jp/dp/" + asin
    print( url )
    bs = get_page_bs( d, url )
    
    seller_id = ""
    price = 0
    
    datas = bs.find_all( 'div', class_ = "tabular-buybox-text" )
    for data in datas:
        if data.find('a') is not None:
            seller_url = data.find('a').get('href')
            if 'seller=' in seller_url:
                index = seller_url.find('seller=')
                seller_id = seller_url[index+7:index+21]
                
            break
    
    #return [ url, seller_id, price ]
    return [asin,"カート販売者",price,seller_id]

In [47]:
# セラーIDから電話番号を収集しにいく
# https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller="{セラーID}
def get_seller_detail( driver, seller_id ):
    seller_url = "https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=" + seller_id
    print( seller_url )
    bs = get_page_bs( d, seller_url )
    for item in bs.find_all( 'div', class_ = "a-column a-span6" ):
        if item.find( 'h3' ) is None:
            continue
        name = ""
        phone = ""
        address = ""
        rep = ""
        store = ""
        for data in item.find_all( 'li' ):
            if "販売業者" in data.text:
                name = data.text[data.text.find(':')+1:]
            if "お問い合わせ先電話番号" in data.text:
                phone = data.text[data.text.find(':')+1:]
            if "住所" in data.text:
                address = data.text[data.text.find(':')+1:]
            if "運営責任者名" in data.text:
                rep = data.text[data.text.find(':')+1:]
            if "店舗名" in data.text:
                store = data.text[data.text.find(':')+1:]
        return [seller_url,name,phone,address,rep,store]
    return [seller_url]

In [48]:
def write_spread_sheet( ws, data_array, index ):
    ws.insert_row(data_array, index)

In [49]:
asins = get_asin()
d = get_driver()

write_spread_sheet( ws_write, ["ASIN","カート販売者か","金額","セラーID","販売者URL","販売者法人名","販売者電話番号","販売者住所","販売代表者名","販売者名"], start_index )
start_index += 1

for asin in asins:
    datas = [ get_cart_seller( d, asin ) ]
    datas.extend ( get_all_sellers( d, asin ) )
    for i in range(0,len(datas)):
        if len( datas[i][3] ) > 0:
            datas[i].extend( get_seller_detail( d, datas[i][3] ) )
        else:
            datas[i].extend( ["https://www.amazon.co.jp/gp/help/customer/display.html?nodeId=202008070","Amazon公式"] )
    
    for data in datas:
        for item in data:
            item = r(item)
    
    for data in datas:
        write_spread_sheet( ws_write, data, start_index )
        start_index += 1

print( "終了" )

https://www.amazon.co.jp/dp/B08Z7N1X83
https://www.amazon.co.jp/gp/product/ajax?asin=B08Z7N1X83&pc=dp&isonlyrenderofferlist=true&pageno=1&experienceId=aodAjaxMain
https://www.amazon.co.jp/gp/product/ajax?asin=B08Z7N1X83&pc=dp&isonlyrenderofferlist=true&pageno=2&experienceId=aodAjaxMain
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AA8TERWRG15GI&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3PL8DDOHAWMLP
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1QCFTKDLRP0RW
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A29LA69IJ0C3QK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A33UDEQFJ4ULS1
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3UIFQMG7D5LMU
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2EYMTTVU96IQK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1EG6IWPQIUZXZ
https://www.amazon.co.jp/dp/B08Z6YRYY6
https://www.amazon.co.jp/gp/product/ajax?asin=B08Z6YRYY6&pc=dp&isonlyrenderofferli

https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2FSULXQNMCGVP
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A6KIO6CIHJJXR&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2QC3QIN062Q5R
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A29LA69IJ0C3QK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3W0X2W5D6AOK6
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2XHZ0JJYJ6E29
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2FHZJ8UZNJN4Q
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3VDY80MQO26TC
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3TCD1E138OLO1
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3PPPHIJCUT54U
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3UIFQMG7D5LMU
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1N29TMPEM5I21
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1EUD5ZBW97AUJ
https://www.amazon.co.jp/sp?is-secure=

https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A34VNVIE2OUU6Z
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A252ETRMXVJC2L
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A37AHJR2UK57OI
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2C77P2JI1EATA
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1EG6IWPQIUZXZ
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2T98KTZIR9O37
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A29LA69IJ0C3QK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A38IZV8A8974CF
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AN5YZMYX29CE&i
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1VW128WHS5SGB
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AKXSIN4OT6BDY&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A304IZ33IVBS9J
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3W0X2W5D6AOK6
https://www.amazon.co.jp/sp?is-secure=

https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A29LA69IJ0C3QK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3JWWUNODUUB4X
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1EG6IWPQIUZXZ
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2XHZ0JJYJ6E29
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3W0X2W5D6AOK6
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A28V5TPDD5X2QZ
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3UIFQMG7D5LMU
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2E8DV3QEWQEHB
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1D0JQCGPAOB7P
https://www.amazon.co.jp/dp/B07K8WHCLK
https://www.amazon.co.jp/gp/product/ajax?asin=B07K8WHCLK&pc=dp&isonlyrenderofferlist=true&pageno=1&experienceId=aodAjaxMain
https://www.amazon.co.jp/gp/product/ajax?asin=B07K8WHCLK&pc=dp&isonlyrenderofferlist=true&pageno=2&experienceId=aodAjaxMain
https://www.amazon.co.jp/gp/product/ajax?asin=B

https://www.amazon.co.jp/gp/product/ajax?asin=B07MNW1G95&pc=dp&isonlyrenderofferlist=true&pageno=2&experienceId=aodAjaxMain
https://www.amazon.co.jp/gp/product/ajax?asin=B07MNW1G95&pc=dp&isonlyrenderofferlist=true&pageno=3&experienceId=aodAjaxMain
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2FSULXQNMCGVP
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A33UDEQFJ4ULS1
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A15FBM6XJ8FX23
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2XHZ0JJYJ6E29
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A29LA69IJ0C3QK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3PPPHIJCUT54U
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AMQJVHU9NX7NQ&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3UIFQMG7D5LMU
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2PBKZI2AOBOL0
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1EUD5ZBW97AUJ
https://www.

https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=APYDS8F2K0D1O&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A243UMV41IVD3M
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1D0JQCGPAOB7P
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1KA3AJ26QA42C
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2OMIKO2VG2QJF
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3G240G9DNPD3S
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3I59M9BOKZ0SF
https://www.amazon.co.jp/dp/B000FQSV78
https://www.amazon.co.jp/gp/product/ajax?asin=B000FQSV78&pc=dp&isonlyrenderofferlist=true&pageno=1&experienceId=aodAjaxMain
https://www.amazon.co.jp/gp/product/ajax?asin=B000FQSV78&pc=dp&isonlyrenderofferlist=true&pageno=2&experienceId=aodAjaxMain
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1D0JQCGPAOB7P
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A304IZ33IVBS9J
https://www.amazon.co.jp/sp?is-secure=true&&ie=

https://www.amazon.co.jp/gp/product/ajax?asin=B007IECDTW&pc=dp&isonlyrenderofferlist=true&pageno=1&experienceId=aodAjaxMain
https://www.amazon.co.jp/gp/product/ajax?asin=B007IECDTW&pc=dp&isonlyrenderofferlist=true&pageno=2&experienceId=aodAjaxMain
https://www.amazon.co.jp/gp/product/ajax?asin=B007IECDTW&pc=dp&isonlyrenderofferlist=true&pageno=3&experienceId=aodAjaxMain
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1PMR9C4RAMGKP
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2FSULXQNMCGVP
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A243UMV41IVD3M
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=ANJQ2FFR2LBT8&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3FINXIZGG7R34
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AJG9NHMS8L7QK&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AED21HEND7FER&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A194HIXKXUWUOQ
https://www.amazon.co.jp/sp?is-secur

https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A29LA69IJ0C3QK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2XHZ0JJYJ6E29
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3PPPHIJCUT54U
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3UIFQMG7D5LMU
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1N29TMPEM5I21
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2R6W8CUS7GNQ5
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AJG9NHMS8L7QK&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1D0JQCGPAOB7P
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A36RMAK13LD1IC
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2QC3QIN062Q5R
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A37AHJR2UK57OI
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2OMIKO2VG2QJF
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3I59M9BOKZ0SF
https://www.amazon.co.jp/sp?is-secure=

https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2XHZ0JJYJ6E29
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AEEX6DFO66EHT&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A30HAZSS2VPXOR
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=AMQJVHU9NX7NQ&
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3UIFQMG7D5LMU
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2EYMTTVU96IQK
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3I59M9BOKZ0SF
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1D0JQCGPAOB7P
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1EG6IWPQIUZXZ
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A1EUD5ZBW97AUJ
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A2OMIKO2VG2QJF
https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=A3G240G9DNPD3S
https://www.amazon.co.jp/dp/B078WS8TVY
https://www.amazon.co.jp/gp/product/ajax?asin=B078WS8TVY&pc=dp&isonlyrend

In [41]:
### パラメータ ###
start_index = 110
col = 0
ws_write = connect_gspread(jsonf,spread_sheet_key,'自動更新')
ws_read = connect_gspread(jsonf,spread_sheet_key,'監視対象ASIN')